In [1]:
!pip install beautifulsoup4 requests

import requests
from bs4 import BeautifulSoup


target_url = "https://csie.asia.edu.tw/zh_tw/TeacherIntroduction/Full_time_faculty"
headers = {"User-Agent": "Magic Browser"}


response = requests.get(target_url, headers=headers)
response.encoding = 'utf-8'
html = BeautifulSoup(response.text, "html.parser")


teacher_blocks = html.find_all("li")

faculty_list = []
current_data = {}


for item in teacher_blocks:
    content = item.get_text(strip=True)

    if content.startswith("姓名"):
        if current_data:
            faculty_list.append(current_data)
        current_data = {}

    if not current_data and not content.startswith("姓名"):
        continue

    for label in ["姓名", "職稱", "學歷", "研究領域"]:
        if content.startswith(label):
            delimiter = "：" if "：" in content else ":"
            _, value = content.split(delimiter, 1)
            current_data[label] = value.strip()
            break


if current_data:
    faculty_list.append(current_data)

output_file = "au_csie_teachers.txt"
with open(output_file, "w", encoding="utf-8") as file:
    for person in faculty_list:
        for field in ["姓名", "職稱", "學歷", "研究領域"]:
            file.write(f"{field}：{person.get(field, '')}\n")
        file.write("=" * 40 + "\n")
